In [64]:
import numpy as np
import pandas as pd
import math
import statistics
import matplotlib.pyplot as plt
from transformers import pipeline
from sklearn.model_selection import train_test_split, cross_val_score,KFold, cross_val_predict, GridSearchCV,RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, roc_auc_score, accuracy_score, precision_score, recall_score, f1_score
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier
from sklearn.datasets import make_classification
from sklearn.neural_network import MLPClassifier


In [65]:
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

## Download All Policies - Train Set

In [66]:
##Privacy Policies download:
websites = ['Google','Aws','AliExpress','Meta','TikTok','YouTube','Waze','Wix','Bookings','whatsapp'
            ,'apple','wolt','Visa','Mastercard','AirBNB','uber','Spotify','samsung','Wordpress','instagram'
            ,'MacDonalds','FDA','Oracle','Zara','cocacola','Xiaomi','Nasdaq','Walmart'
           ,'AirCanada','Lufthansa','shopify','Netflix','adobe','Starbucks','Shoppers','Decathlon','waltdisney'
            ,'AmericanEagle','lululemon','SAP','JetBrains','MySQLCode','Cadens','EpicGames'
            ,'unitedHealthGroup','Slack','SalesForce','JPMorgan','JohnsonAndJohnson']

sequence_to_classify = []

for i in websites:
    try:
        with open((i+".txt"), "r") as f:
            i = f.readlines()
            sequence_to_classify.append(i)
    except:
        with open((i+".txt"), "r", encoding='cp1252') as f:
            i = f.readlines()
            sequence_to_classify.append(i)

### Train Set Vectors:

In [67]:
data_security_train = [[1,3],[1,3,4],[3],[2],[3],[3],[3],[3,4],[1],[3],[1],[1],[1],[1],[1],[2],[1],[1],[3],[2]
                       ,[3],[1],[1],[1],[1],[1,2],[1],[1],[1],[2],[1],[3],[2],[2],[3],[1],[3],[3],[3],[1,2]
                       ,[1,2],[3],[3],[3],[3],[2],[2],[3],[3]]

In [68]:
data_security_train_2FA = [1,1,0,0,0,0,0,0,1,0,1,1,1,1,1,0,1,1,0,0,0,1,1,1,1,1,1,1,1,0,1,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0]
data_security_train_GDPR = [0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,1,1,0,0,0,0,0,1,1,0,0,0,0,1,1,0,0]
data_security_train_other =[1,1,1,0,1,1,1,1,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,1,0,0,1,1,1,1,0,0,1,1,1]
#[1,1,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,1,0,0,0,0,1,1,1,0,0,0,0]
# data_security_train_No = [0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,1]
data_security_train_PCI_DSS = [0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]

### Data Security Parameter

In [69]:
##data_security Matrix - 2FA [1]:

Phrases = ['Privacy Policy'
           ,'we are using 2FA method'
           ,'we are not using 2FA method']

##Matrix creation:
matrix_data_security_ML_2FA = pd.DataFrame(columns = Phrases)
matrix_data_security_ML_2FA['Privacy Policy'] = websites
matrix_data_security_ML_2FA.set_index('Privacy Policy', inplace=True)

##Filling the matrix:
line = 0
for j in sequence_to_classify:
    for i in matrix_data_security_ML_2FA:
        matrix_data_security_ML_2FA.loc[websites[line],i] = classifier(str(j), str(i))["scores"][0]
    line += 1

matrix_data_security_ML_2FA

,we are using 2FA method,we are not using 2FA method
Privacy Policy,,
Google,0.907859,0.432299
Aws,0.748062,0.53815
AliExpress,0.762992,0.177296
Meta,0.911825,0.692897
TikTok,0.767259,0.595065
YouTube,0.807246,0.276014
Waze,0.738516,0.202823
Wix,0.928329,0.547194
Bookings,0.969526,0.747512


### Test The Model - Privacy Policy

In [70]:
# Manual division so we will have all options (1/0) at both groups
X_train = matrix_data_security_ML_2FA.iloc[:35,:] 
X_test = matrix_data_security_ML_2FA.iloc[34:,:]
y_train = data_security_train_2FA[:35]
y_test = data_security_train_2FA[34:]

##Decission Tree Classifaier
regressor_tree = DecisionTreeClassifier()
regressor_tree = regressor_tree.fit(X_train, y_train)
pred_tree = regressor_tree.predict(X_test)
print('Decission Tree Classifaier:', pred_tree)
print(classification_report(y_test, pred_tree))

##Random Forest Classifaier
regressor_forest = RandomForestClassifier()
regressor_forest = regressor_forest.fit(X_train, y_train)
pred_forest = regressor_forest.predict(X_test).round()
print('Random Forest Classifaier:', pred_forest)
print(classification_report(y_test, pred_forest))

##Logistic Reg Classifier
clf = LogisticRegression()
clf.fit(X_train, y_train)
pred_clf = clf.predict(X_test)
print('Logistic Reg Classifier:', pred_clf)
print(classification_report(y_test, pred_clf))

##SVM
SVM = SVC(kernel='linear')
SVM = SVM.fit(X_train, y_train)
pred_SVM = SVM.predict(X_test)
print('SVM Classifier:', pred_SVM)
print(classification_report(y_test, pred_SVM))

##KNN
KNN = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p=2)
KNN = KNN.fit(X_train, y_train)
pred_KNN = KNN.predict(X_test)
print('KNN Classifier:', pred_KNN)
print(classification_report(y_test, pred_KNN))

##neural_network
neural_network = MLPClassifier(random_state=1, max_iter=300)
neural_network = neural_network.fit(X_train, y_train)
pred_neural_network = neural_network.predict(X_test)
print('neural_network Classifier:', pred_neural_network)
print(classification_report(y_test, pred_neural_network))


Decission Tree Classifaier: [0 1 1 0 1 1 1 1 0 1 0 1 1 1 1]
              precision    recall  f1-score   support

           0       1.00      0.33      0.50        12
           1       0.27      1.00      0.43         3

    accuracy                           0.47        15
   macro avg       0.64      0.67      0.46        15
weighted avg       0.85      0.47      0.49        15

Random Forest Classifaier: [0 0 1 0 1 1 1 0 0 1 0 1 1 1 1]
              precision    recall  f1-score   support

           0       0.83      0.42      0.56        12
           1       0.22      0.67      0.33         3

    accuracy                           0.47        15
   macro avg       0.53      0.54      0.44        15
weighted avg       0.71      0.47      0.51        15

Logistic Reg Classifier: [1 1 1 1 1 1 1 1 0 1 1 1 1 1 1]
              precision    recall  f1-score   support

           0       1.00      0.08      0.15        12
           1       0.21      1.00      0.35         3

    ac

/Users/yuvalba/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/yuvalba/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/yuvalba/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/yuvalb

In [71]:
pred_2FA = pred_tree

In [72]:
##data_security Matrix - GDPR [2]:

Phrases = ['Privacy Policy'
           ,'we are follow GDPR'
           ,'we are not follow GDPR'
           ,'our privacy policy is Under General Data Protection Regulation']

##Matrix creation:
matrix_data_security_ML_GDPR = pd.DataFrame(columns = Phrases)
matrix_data_security_ML_GDPR['Privacy Policy'] = websites
matrix_data_security_ML_GDPR.set_index('Privacy Policy', inplace=True)

##Filling the matrix:
line = 0
for j in sequence_to_classify:
    for i in matrix_data_security_ML_GDPR:
        matrix_data_security_ML_GDPR.loc[websites[line],i] = classifier(str(j), str(i))["scores"][0]
    line += 1
    
matrix_data_security_ML_GDPR

,we are follow GDPR,we are not follow GDPR,our privacy policy is Under General Data Protection Regulation
Privacy Policy,,,
Google,0.948273,0.69066,0.892894
Aws,0.885237,0.751424,0.911755
AliExpress,0.82584,0.229803,0.95423
Meta,0.969088,0.827191,0.94612
TikTok,0.88553,0.731645,0.881789
YouTube,0.935752,0.402478,0.946355
Waze,0.8668,0.502773,0.91155
Wix,0.873112,0.448174,0.867495
Bookings,0.958984,0.824295,0.952983


In [73]:
# Manual division so we will have all options (1/0) at both groups
X_train = matrix_data_security_ML_GDPR.iloc[:35,:] 
X_test = matrix_data_security_ML_GDPR.iloc[34:,:]
y_train = data_security_train_GDPR[:35]
y_test = data_security_train_GDPR[34:]

##Decission Tree Classifaier
regressor_tree = DecisionTreeClassifier()
regressor_tree = regressor_tree.fit(X_train, y_train)
pred_tree = regressor_tree.predict(X_test)
print('Decission Tree Classifaier:', pred_tree)
print(classification_report(y_test, pred_tree))

##Random Forest Classifaier
regressor_forest = RandomForestClassifier()
regressor_forest = regressor_forest.fit(X_train, y_train)
pred_forest = regressor_forest.predict(X_test).round()
print('Random Forest Classifaier:', pred_forest)
print(classification_report(y_test, pred_forest))

##Logistic Reg Classifier
clf = LogisticRegression()
clf.fit(X_train, y_train)
pred_clf = clf.predict(X_test)
print('Logistic Reg Classifier:', pred_clf)
print(classification_report(y_test, pred_clf))

##SVM
SVM = SVC(kernel='linear')
SVM = SVM.fit(X_train, y_train)
pred_SVM = SVM.predict(X_test)
print('SVM Classifier:', pred_SVM)
print(classification_report(y_test, pred_SVM))

##KNN
KNN = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p=2)
KNN = KNN.fit(X_train, y_train)
pred_KNN = KNN.predict(X_test)
print('KNN Classifier:', pred_KNN)
print(classification_report(y_test, pred_KNN))

##neural_network
neural_network = MLPClassifier(random_state=1, max_iter=300)
neural_network = neural_network.fit(X_train, y_train)
pred_neural_network = neural_network.predict(X_test)
print('neural_network Classifier:', pred_neural_network)
print(classification_report(y_test, pred_neural_network))

Decission Tree Classifaier: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
              precision    recall  f1-score   support

           0       0.73      1.00      0.85        11
           1       0.00      0.00      0.00         4

    accuracy                           0.73        15
   macro avg       0.37      0.50      0.42        15
weighted avg       0.54      0.73      0.62        15

Random Forest Classifaier: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
              precision    recall  f1-score   support

           0       0.73      1.00      0.85        11
           1       0.00      0.00      0.00         4

    accuracy                           0.73        15
   macro avg       0.37      0.50      0.42        15
weighted avg       0.54      0.73      0.62        15

Logistic Reg Classifier: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
              precision    recall  f1-score   support

           0       0.73      1.00      0.85        11
           1       0.00      0.00      0.00         4

    ac

/Users/yuvalba/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/yuvalba/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/yuvalba/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/yuvalb

neural_network Classifier: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
              precision    recall  f1-score   support

           0       0.73      1.00      0.85        11
           1       0.00      0.00      0.00         4

    accuracy                           0.73        15
   macro avg       0.37      0.50      0.42        15
weighted avg       0.54      0.73      0.62        15



/Users/yuvalba/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/yuvalba/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/yuvalba/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [74]:
pred_GDPR = pred_tree

In [75]:
##data_security Matrix - other protocols [3]:

Phrases = ['Privacy Policy'
           ,'we secure your data with other protocols'
           ,'your data is secure by different protocols'
           ,'we do not secure your data']

##Matrix creation:
matrix_data_security_ML_other = pd.DataFrame(columns = Phrases)
matrix_data_security_ML_other['Privacy Policy'] = websites
matrix_data_security_ML_other.set_index('Privacy Policy', inplace=True)

##Filling the matrix:
line = 0
for j in sequence_to_classify:
    for i in matrix_data_security_ML_other:
        matrix_data_security_ML_other.loc[websites[line],i] = classifier(str(j), str(i))["scores"][0]
    line += 1
    
matrix_data_security_ML_other

,we secure your data with other protocols,your data is secure by different protocols,we do not secure your data
Privacy Policy,,,
Google,0.903117,0.919415,0.700113
Aws,0.940654,0.938267,0.831412
AliExpress,0.73657,0.343118,0.183208
Meta,0.917371,0.929208,0.871657
TikTok,0.877305,0.885334,0.804881
YouTube,0.959781,0.9621,0.642862
Waze,0.932786,0.916489,0.669012
Wix,0.959305,0.95222,0.691161
Bookings,0.983538,0.977893,0.959422


In [76]:
# Manual division so we will have all options (1/0) at both groups
X_train = matrix_data_security_ML_other.iloc[:35,:] 
X_test = matrix_data_security_ML_other.iloc[34:,:]
y_train = data_security_train_other[:35]
y_test = data_security_train_other[34:]

##Decission Tree Classifaier
regressor_tree = DecisionTreeClassifier()
regressor_tree = regressor_tree.fit(X_train, y_train)
pred_tree = regressor_tree.predict(X_test)
print('Decission Tree Classifaier:', pred_tree)
print(classification_report(y_test, pred_tree))

##Random Forest Classifaier
regressor_forest = RandomForestClassifier()
regressor_forest = regressor_forest.fit(X_train, y_train)
pred_forest = regressor_forest.predict(X_test).round()
print('Random Forest Classifaier:', pred_forest)
print(classification_report(y_test, pred_forest))

##Logistic Reg Classifier
clf = LogisticRegression()
clf.fit(X_train, y_train)
pred_clf = clf.predict(X_test)
print('Logistic Reg Classifier:', pred_clf)
print(classification_report(y_test, pred_clf))

##SVM
SVM = SVC(kernel='linear')
SVM = SVM.fit(X_train, y_train)
pred_SVM = SVM.predict(X_test)
print('SVM Classifier:', pred_SVM)
print(classification_report(y_test, pred_SVM))

##KNN
KNN = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p=2)
KNN = KNN.fit(X_train, y_train)
pred_KNN = KNN.predict(X_test)
print('KNN Classifier:', pred_KNN)
print(classification_report(y_test, pred_KNN))

##neural_network
neural_network = MLPClassifier(random_state=1, max_iter=300)
neural_network = neural_network.fit(X_train, y_train)
pred_neural_network = neural_network.predict(X_test)
print('neural_network Classifier:', pred_neural_network)
print(classification_report(y_test, pred_neural_network))

Decission Tree Classifaier: [1 1 1 0 0 0 1 1 1 1 1 0 1 0 0]
              precision    recall  f1-score   support

           0       0.50      0.60      0.55         5
           1       0.78      0.70      0.74        10

    accuracy                           0.67        15
   macro avg       0.64      0.65      0.64        15
weighted avg       0.69      0.67      0.67        15

Random Forest Classifaier: [1 1 0 0 0 0 1 1 0 1 1 0 1 0 0]
              precision    recall  f1-score   support

           0       0.38      0.60      0.46         5
           1       0.71      0.50      0.59        10

    accuracy                           0.53        15
   macro avg       0.54      0.55      0.52        15
weighted avg       0.60      0.53      0.55        15

Logistic Reg Classifier: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
              precision    recall  f1-score   support

           0       0.33      1.00      0.50         5
           1       0.00      0.00      0.00        10

    ac

/Users/yuvalba/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/yuvalba/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/yuvalba/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/yuvalb

In [77]:
pred_other = pred_forest

In [78]:
# ##data_security Matrix - no secure [4]:

# Phrases = ['Privacy Policy'
#            ,'we secure your data with other protocols'
#            ,'your data is secure by different protocols'
#            ,'we do not secure your data']

# ##Matrix creation:
# matrix_data_security_ML_No = pd.DataFrame(columns = Phrases)
# matrix_data_security_ML_No['Privacy Policy'] = websites
# matrix_data_security_ML_No.set_index('Privacy Policy', inplace=True)

# ##Filling the matrix:
# line = 0
# for j in sequence_to_classify:
#     for i in matrix_data_security_ML_No:
#         matrix_data_security_ML_No.loc[websites[line],i] = classifier(str(j), str(i))["scores"][0]
#     line += 1
    
# matrix_data_security_ML_No

In [79]:
# # Manual division so we will have all options (1/0) at both groups
# X_train = matrix_data_security_ML_No.iloc[:23,:] 
# X_test = matrix_data_security_ML_No.iloc[22:,:]
# y_train = data_security_train_No[:23]
# y_test = data_security_train_No[22:]

# ##Decission Tree Classifaier
# regressor_tree = DecisionTreeClassifier()
# regressor_tree = regressor_tree.fit(X_train, y_train)
# pred_tree = regressor_tree.predict(X_test)
# print('Decission Tree Classifaier:', pred_tree)
# print(classification_report(y_test, pred_tree))

# ##Random Forest Classifaier
# regressor_forest = RandomForestClassifier()
# regressor_forest = regressor_forest.fit(X_train, y_train)
# pred_forest = regressor_forest.predict(X_test).round()
# print('Random Forest Classifaier:', pred_forest)
# print(classification_report(y_test, pred_forest))

# ##Logistic Reg Classifier
# clf = LogisticRegression()
# clf.fit(X_train, y_train)
# pred_clf = clf.predict(X_test)
# print('Logistic Reg Classifier:', pred_clf)
# print(classification_report(y_test, pred_clf))

# ##SVM
# SVM = SVC(kernel='linear')
# SVM = SVM.fit(X_train, y_train)
# pred_SVM = SVM.predict(X_test)
# print('SVM Classifier:', pred_SVM)
# print(classification_report(y_test, pred_SVM))

# ##KNN
# KNN = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p=2)
# KNN = KNN.fit(X_train, y_train)
# pred_KNN = KNN.predict(X_test)
# print('KNN Classifier:', pred_KNN)
# print(classification_report(y_test, pred_KNN))

# ##neural_network
# neural_network = MLPClassifier(random_state=1, max_iter=300)
# neural_network = neural_network.fit(X_train, y_train)
# pred_neural_network = neural_network.predict(X_test)
# print('neural_network Classifier:', pred_neural_network)
# print(classification_report(y_test, pred_neural_network))

In [80]:
##data_security Matrix - according to PCI DSS [5]:

Phrases = ['Privacy Policy'
           ,'we are follow PCI DSS'
           ,'we are not follow PCI DSS']

##Matrix creation:
matrix_data_security_ML_PCI_DSS = pd.DataFrame(columns = Phrases)
matrix_data_security_ML_PCI_DSS['Privacy Policy'] = websites
matrix_data_security_ML_PCI_DSS.set_index('Privacy Policy', inplace=True)

##Filling the matrix:
line = 0
for j in sequence_to_classify:
    for i in matrix_data_security_ML_PCI_DSS:
        matrix_data_security_ML_PCI_DSS.loc[websites[line],i] = classifier(str(j), str(i))["scores"][0]
    line += 1
    
matrix_data_security_ML_PCI_DSS

,we are follow PCI DSS,we are not follow PCI DSS
Privacy Policy,,
Google,0.913529,0.441668
Aws,0.7131,0.529085
AliExpress,0.740179,0.23405
Meta,0.934205,0.903889
TikTok,0.777295,0.666322
YouTube,0.862218,0.491663
Waze,0.823254,0.315871
Wix,0.670935,0.271942
Bookings,0.941627,0.604615


In [81]:
# X_train, X_test, y_train, y_test = train_test_split(matrix_policy_change, policy_change_train
#                                                     , test_size=0.5, random_state=42)

# Manual division so we will have all options (1/0) at both groups
X_train = matrix_data_security_ML_PCI_DSS.iloc[:35,:] 
X_test = matrix_data_security_ML_PCI_DSS.iloc[34:,:]
y_train = data_security_train_PCI_DSS[:35]
y_test = data_security_train_PCI_DSS[34:]

##Decission Tree Classifaier
regressor_tree = DecisionTreeClassifier()
regressor_tree = regressor_tree.fit(X_train, y_train)
pred_tree = regressor_tree.predict(X_test)
print('Decission Tree Classifaier:', pred_tree)
print(classification_report(y_test, pred_tree))

##Random Forest Classifaier
regressor_forest = RandomForestClassifier()
regressor_forest = regressor_forest.fit(X_train, y_train)
pred_forest = regressor_forest.predict(X_test).round()
print('Random Forest Classifaier:', pred_forest)
print(classification_report(y_test, pred_forest))

##Logistic Reg Classifier
clf = LogisticRegression()
clf.fit(X_train, y_train)
pred_clf = clf.predict(X_test)
print('Logistic Reg Classifier:', pred_clf)
print(classification_report(y_test, pred_clf))

##SVM
SVM = SVC(kernel='linear')
SVM = SVM.fit(X_train, y_train)
pred_SVM = SVM.predict(X_test)
print('SVM Classifier:', pred_SVM)
print(classification_report(y_test, pred_SVM))

##KNN
KNN = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p=2)
KNN = KNN.fit(X_train, y_train)
pred_KNN = KNN.predict(X_test)
print('KNN Classifier:', pred_KNN)
print(classification_report(y_test, pred_KNN))

##neural_network
neural_network = MLPClassifier(random_state=1, max_iter=300)
neural_network = neural_network.fit(X_train, y_train)
pred_neural_network = neural_network.predict(X_test)
print('neural_network Classifier:', pred_neural_network)
print(classification_report(y_test, pred_neural_network))



Decission Tree Classifaier: [0 0 0 0 0 0 0 1 1 0 0 0 0 0 0]
              precision    recall  f1-score   support

           0       1.00      0.87      0.93        15
           1       0.00      0.00      0.00         0

    accuracy                           0.87        15
   macro avg       0.50      0.43      0.46        15
weighted avg       1.00      0.87      0.93        15

Random Forest Classifaier: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        15

    accuracy                           1.00        15
   macro avg       1.00      1.00      1.00        15
weighted avg       1.00      1.00      1.00        15

Logistic Reg Classifier: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        15

    accuracy                           1.00        15
   macro avg       1.00      1.00      1.00        15
weight

/Users/yuvalba/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/yuvalba/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/yuvalba/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/yuvalba/opt/anaconda3/lib/pyth

neural_network Classifier: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        15

    accuracy                           1.00        15
   macro avg       1.00      1.00      1.00        15
weighted avg       1.00      1.00      1.00        15



In [82]:
pred_PCI_DSS = pred_clf

In [83]:
#Data Security Parameter:
index = 0
TP = 0
FP = 0
FN = 0
TN = 0
for i in data_security_train_2FA[34:]:
    if pred_2FA[index]==1 and data_security_train_2FA[index+34]==1:
        TP +=1
    elif pred_2FA[index]==0 and data_security_train_2FA[index+34]==0:
        TN +=1
    elif pred_2FA[index]==0 and data_security_train_2FA[index+34]==1:
        FN +=1
    else:
        FP +=1
    index +=1

index = 0
for i in data_security_train_GDPR[34:]:
    if pred_GDPR[index]==1 and data_security_train_GDPR[index+34]==1:
        TP +=1
    elif pred_GDPR[index]==0 and data_security_train_GDPR[index+34]==0:
        TN +=1
    elif pred_GDPR[index]==0 and data_security_train_GDPR[index+34]==1:
        FN +=1
    else:
        FP +=1
    index +=1
    
index = 0
for i in data_security_train_other[34:]:
    if pred_other[index]==1 and data_security_train_other[index+34]==1:
        TP +=1
    elif pred_other[index]==0 and data_security_train_other[index+34]==0:
        TN +=1
    elif pred_other[index]==0 and data_security_train_other[index+34]==1:
        FN +=1
    else:
        FP +=1
    index +=1
    
# index = 0    
# for i in data_security_train_No[34:]:
#     if pred_No[index]==1 and data_security_train_No[index+34]==1:
#         TP +=1
#     elif pred_No[index]==0 and data_security_train_No[index+34]==0:
#         TN +=1
#     elif pred_No[index]==0 and data_security_train_No[index+34]==1:
#         FN +=1
#     else:
#         FP +=1
#     index +=1
    
index = 0    
for i in data_security_train_PCI_DSS[34:]:
    if pred_PCI_DSS[index]==1 and data_security_train_PCI_DSS[index+34]==1:
        TP +=1
    elif pred_PCI_DSS[index]==0 and data_security_train_PCI_DSS[index+34]==0:
        TN +=1
    elif pred_PCI_DSS[index]==0 and data_security_train_PCI_DSS[index+34]==1:
        FN +=1
    else:
        FP +=1
    index +=1
    
recall = TP/(TP+FN)
Precision = TP/(TP+FP)
Accuracy = (TP+TN)/(TP+FP+FN+TN) 
F1 = (TP)/(TP+(0.5*(FP+FN)))

print("recall:" , recall)
print("Precision:" , Precision)
print("Accuracy:" , Accuracy)
print("F1 Score:" , F1)

recall: 0.47058823529411764
Precision: 0.4444444444444444
Accuracy: 0.6833333333333333
F1 Score: 0.45714285714285713


In [32]:
##Privacy Policy Change Matrix:

##Matrix creation:
matrix_policy_change_Median_2FA = matrix_data_security_ML_2FA.transpose()
    
#add the avg and median rows:
matrix_policy_change_Median_2FA.loc['Median'] =  matrix_policy_change_Median_2FA.median()
matrix_policy_change_Median_2FA.loc['Median']['Privacy Policy'] = 'Median'

policy_change_train_pred_Median_2FA = []
websites_index = 0

for i in matrix_policy_change_Median_2FA:
    num = matrix_policy_change_Median_2FA[websites[websites_index]].loc['Median']
    if num >0.5:
        policy_change_train_pred_Median_2FA.append(1)
    else:
        policy_change_train_pred_Median_2FA.append(0)
    websites_index += 1
 
print(policy_change_train_pred_Median_2FA)

[1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1]


In [34]:
##Privacy Policy Change Matrix:

##Matrix creation:
matrix_policy_change_Median_GDPR = matrix_data_security_ML_GDPR.transpose()
    
#add the avg and median rows:
matrix_policy_change_Median_GDPR.loc['Median'] =  matrix_policy_change_Median_GDPR.median()
matrix_policy_change_Median_GDPR.loc['Median']['Privacy Policy'] = 'Median'

policy_change_train_pred_Median_GDPR = []
websites_index = 0

for i in matrix_policy_change_Median_GDPR:
    num = matrix_policy_change_Median_GDPR[websites[websites_index]].loc['Median']
    if num >0.93:
        policy_change_train_pred_Median_GDPR.append(1)
    else:
        policy_change_train_pred_Median_GDPR.append(0)
    websites_index += 1
 
print(policy_change_train_pred_Median_GDPR)

[0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0]


In [36]:
##Privacy Policy Change Matrix:

##Matrix creation:
matrix_policy_change_Median_other = matrix_data_security_ML_other.transpose()
    
#add the avg and median rows:
matrix_policy_change_Median_other.loc['Median'] =  matrix_policy_change_Median_other.median()
matrix_policy_change_Median_other.loc['Median']['Privacy Policy'] = 'Median'

policy_change_train_pred_Median_other = []
websites_index = 0

for i in matrix_policy_change_Median_other:
    num = matrix_policy_change_Median_other[websites[websites_index]].loc['Median']
    if num >0.5:
        policy_change_train_pred_Median_other.append(1)
    else:
        policy_change_train_pred_Median_other.append(0)
    websites_index += 1
 
print(policy_change_train_pred_Median_other)

[1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [40]:
##Privacy Policy Change Matrix:

##Matrix creation:
matrix_policy_change_Median_PCI_DSS = matrix_data_security_ML_PCI_DSS.transpose()
    
#add the avg and median rows:
matrix_policy_change_Median_PCI_DSS.loc['Median'] =  matrix_policy_change_Median_PCI_DSS.median()
matrix_policy_change_Median_PCI_DSS.loc['Median']['Privacy Policy'] = 'Median'

policy_change_train_pred_Median_PCI_DSS = []
websites_index = 0

for i in matrix_policy_change_Median_PCI_DSS:
    num = matrix_policy_change_Median_PCI_DSS[websites[websites_index]].loc['Median']
    if num >0.85:
        policy_change_train_pred_Median_PCI_DSS.append(1)
    else:
        policy_change_train_pred_Median_PCI_DSS.append(0)
    websites_index += 1
 
print(policy_change_train_pred_Median_PCI_DSS)

[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [47]:
#Data Security Parameter:
index = 0
TP = 0
FP = 0
FN = 0
TN = 0
for i in policy_change_train_pred_Median_2FA:
    if policy_change_train_pred_Median_2FA[index]==1 and data_security_train_2FA[index]==1:
        TP +=1
    elif policy_change_train_pred_Median_2FA[index]==0 and data_security_train_2FA[index]==0:
        TN +=1
    elif policy_change_train_pred_Median_2FA[index]==0 and data_security_train_2FA[index]==1:
        FN +=1
    else:
        FP +=1
    index +=1

index = 0
for i in policy_change_train_pred_Median_GDPR:
    if policy_change_train_pred_Median_GDPR[index]==1 and data_security_train_GDPR[index]==1:
        TP +=1
    elif policy_change_train_pred_Median_GDPR[index]==0 and data_security_train_GDPR[index]==0:
        TN +=1
    elif policy_change_train_pred_Median_GDPR[index]==0 and data_security_train_GDPR[index]==1:
        FN +=1
    else:
        FP +=1
    index +=1
    
index = 0
for i in policy_change_train_pred_Median_other:
    if policy_change_train_pred_Median_other[index]==1 and data_security_train_other[index]==1:
        TP +=1
    elif policy_change_train_pred_Median_other[index]==0 and data_security_train_other[index]==0:
        TN +=1
    elif policy_change_train_pred_Median_other[index]==0 and data_security_train_other[index]==1:
        FN +=1
    else:
        FP +=1
    index +=1
    
index = 0    
for i in policy_change_train_pred_Median_PCI_DSS:
    if policy_change_train_pred_Median_PCI_DSS[index]==1 and data_security_train_PCI_DSS[index]==1:
        TP +=1
    elif policy_change_train_pred_Median_PCI_DSS[index]==0 and data_security_train_PCI_DSS[index]==0:
        TN +=1
    elif policy_change_train_pred_Median_PCI_DSS[index]==0 and data_security_train_PCI_DSS[index]==1:
        FN +=1
    else:
        FP +=1
    index +=1
    
recall = TP/(TP+FN)
Precision = TP/(TP+FP)
Accuracy = (TP+TN)/(TP+FP+FN+TN) 
F1 = (TP)/(TP+(0.5*(FP+FN)))

print("recall:" , recall)
print("Precision:" , Precision)
print("Accuracy:" , Accuracy)
print("F1 Score:" , F1)

recall: 0.9107142857142857
Precision: 0.4214876033057851
Accuracy: 0.6173469387755102
F1 Score: 0.576271186440678


In [145]:
data_security_train_pred = []
lst_temp = []
par = [policy_change_train_pred_Median_2FA, policy_change_train_pred_Median_GDPR
       ,policy_change_train_pred_Median_other, policy_change_train_pred_Median_No
       ,policy_change_train_pred_Median_PCI_DSS]

index = 1

for i in range(0,28):
    for j in par:
        if j[i]==1:
            lst_temp.append(index)
        index += 1
    index =1
    data_security_train_pred.append(lst_temp)
    lst_temp = []

index = 0
for i in data_security_train_pred:
    if i == []:
        data_security_train_pred[index] = [4]
    index+=1
print(data_security_train_pred)

[[1, 3], [1, 3], [4], [1, 2, 3, 5], [1, 3], [1, 2, 3], [3], [1, 3], [1, 2, 3], [1, 3], [1, 2, 3], [1, 2, 3, 5], [1, 3], [1, 2, 3], [1, 3], [1, 2, 3], [1, 2, 3, 4], [1, 3], [1, 3], [1, 2, 3], [1, 2, 3, 4, 5], [3], [1, 2, 3], [1, 2, 3], [1, 2, 3, 5], [1, 2, 3], [1, 3], [1, 3]]


In [48]:
# ##Privacy Policy Change Matrix:

# ##Matrix creation:
# matrix_data_security_Comparison = pd.DataFrame(columns = ['Phrase',
#                                                'Google','Aws','AliExpress','Meta','TikTok','YouTube','Waze','Wix','Bookings','whatsapp'
#             ,'apple','wolt','Visa','Mastercard','AirBNB','uber','Spotify','samsung','Wordpress','instagram'
#             ,'MacDonalds','FDA','Oracle','Zara','cocacola','Xiaomi','Nasdaq','Walmart'
#            ,'AirCanada','Lufthansa','shopify','Netflix','adobe','Starbucks','Shoppers','Decathlon','waltdisney'
#             ,'AmericanEagle','lululemon','SAP','JetBrains','MySQLCode','Cadens','EpicGames'
#             ,'unitedHealthGroup','Slack','SalesForce','JPMorgan','JohnsonAndJohnson'])
# matrix_data_security_Comparison['Phrase'] = [['we are using 2FA method','we are not using 2FA method']
#                                  ,['we are follow GDPR','we are not follow GDPR']
#                                  ,['we secure your data with other protocols','we are not secure your data']
#                                  ,['we secure your data','we are not secure your data']
#                                  ,['we are follow PCI DSS','we are not follow PCI DSS']]

# ##Filling the matrix:
# websites_index = 0
# line = 0
# pred = 0
# data_security_train_pred_Comparison = []
# data_security_list_Comparison = []

# for j in sequence_to_classify:
#     for i in matrix_data_security_Comparison["Phrase"]:
#         a = classifier(str(j), str(i))
#         if matrix_data_security_Comparison['Phrase'][line][0] in a["labels"][0]:
#             pred = 1
#         else:
#             pred = 0 
#         if pred == 1:
#             data_security_list_Comparison.append(line+1)
#         line += 1
#     websites_index += 1
#     data_security_train_pred_Comparison.append(data_security_list_Comparison)
#     data_security_list_Comparison = []
#     line = 0

# data_security_train_pred_Comparison

    

[[1, 2, 3, 5],
 [1, 2, 3, 4, 5],
 [1, 2, 3, 4, 5],
 [1, 2, 4, 5],
 [1, 2, 3, 4, 5],
 [1, 2, 3, 4, 5],
 [1, 2, 3, 4, 5],
 [1, 2, 3, 4, 5],
 [1, 2, 3, 4, 5],
 [1, 2, 4],
 [1, 2, 3, 4, 5],
 [1, 2, 3, 4, 5],
 [1, 2, 3, 4, 5],
 [1, 2, 3, 4, 5],
 [1, 2, 3, 4, 5],
 [1, 2, 3, 4, 5],
 [1, 2, 3, 4, 5],
 [1, 2, 3, 4, 5],
 [1, 2, 3, 4],
 [1, 2, 3, 4, 5],
 [1, 2, 3, 4, 5],
 [1, 2, 3, 4, 5],
 [1, 2, 3, 4, 5],
 [1, 2, 3, 4, 5],
 [1, 2, 5],
 [1, 2, 3, 4, 5],
 [1, 2, 3, 4, 5],
 [1, 2, 3, 4, 5],
 [1, 2, 3, 4, 5],
 [1, 2, 3, 4, 5],
 [1, 2, 5],
 [1, 2, 3, 4, 5],
 [1, 2, 3, 4, 5],
 [1, 2, 3, 4, 5],
 [1, 2, 5],
 [1, 2, 3, 4, 5],
 [1, 2, 3, 4, 5],
 [1, 2, 3, 4, 5],
 [1, 2, 3, 4, 5],
 [1, 2, 3, 4, 5],
 [1, 2, 3, 4, 5],
 [1, 2, 3, 5],
 [1, 2, 3, 4, 5],
 [1, 2, 3, 4, 5],
 [1, 2, 3, 4, 5],
 [1, 2, 3, 4, 5],
 [1, 2, 3, 4, 5],
 [1, 2, 3, 5],
 [1, 2, 3, 4, 5]]

In [52]:
##Privacy Policy Change Matrix:

##Matrix creation:
matrix_data_security_Comparison = pd.DataFrame(columns = ['Phrase',
                                               'Google','Aws','AliExpress','Meta','TikTok','YouTube','Waze','Wix','Bookings','whatsapp'
            ,'apple','wolt','Visa','Mastercard','AirBNB','uber','Spotify','samsung','Wordpress','instagram'
            ,'MacDonalds','FDA','Oracle','Zara','cocacola','Xiaomi','Nasdaq','Walmart'
           ,'AirCanada','Lufthansa','shopify','Netflix','adobe','Starbucks','Shoppers','Decathlon','waltdisney'
            ,'AmericanEagle','lululemon','SAP','JetBrains','MySQLCode','Cadens','EpicGames'
            ,'unitedHealthGroup','Slack','SalesForce','JPMorgan','JohnsonAndJohnson'])
matrix_data_security_Comparison['Phrase'] = [['we are using 2FA method','we are not using 2FA method']]

##Filling the matrix:
websites_index = 0
line = 0
pred = 0
data_security_train_pred_Comparison_2FA = []
data_security_list_Comparison = []

for j in sequence_to_classify:
    for i in matrix_data_security_Comparison["Phrase"]:
        a = classifier(str(j), str(i))
        if matrix_data_security_Comparison['Phrase'][line][0] in a["labels"][0]:
            pred = 1
        else:
            pred = 0 
        if pred == 1:
            data_security_list_Comparison.append(line+1)
        line += 1
    websites_index += 1
    data_security_train_pred_Comparison_2FA.append(data_security_list_Comparison)
    data_security_list_Comparison = []
    line = 0

data_security_train_pred_Comparison_2FA

    

[[1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1]]

In [53]:
##Privacy Policy Change Matrix:

##Matrix creation:
matrix_data_security_Comparison = pd.DataFrame(columns = ['Phrase',
                                               'Google','Aws','AliExpress','Meta','TikTok','YouTube','Waze','Wix','Bookings','whatsapp'
            ,'apple','wolt','Visa','Mastercard','AirBNB','uber','Spotify','samsung','Wordpress','instagram'
            ,'MacDonalds','FDA','Oracle','Zara','cocacola','Xiaomi','Nasdaq','Walmart'
           ,'AirCanada','Lufthansa','shopify','Netflix','adobe','Starbucks','Shoppers','Decathlon','waltdisney'
            ,'AmericanEagle','lululemon','SAP','JetBrains','MySQLCode','Cadens','EpicGames'
            ,'unitedHealthGroup','Slack','SalesForce','JPMorgan','JohnsonAndJohnson'])
matrix_data_security_Comparison['Phrase'] = [['we are follow GDPR','we are not follow GDPR']]

##Filling the matrix:
websites_index = 0
line = 0
pred = 0
data_security_train_pred_Comparison_GPDR = []
data_security_list_Comparison = []

for j in sequence_to_classify:
    for i in matrix_data_security_Comparison["Phrase"]:
        a = classifier(str(j), str(i))
        if matrix_data_security_Comparison['Phrase'][line][0] in a["labels"][0]:
            pred = 1
        else:
            pred = 0 
        if pred == 1:
            data_security_list_Comparison.append(line+1)
        line += 1
    websites_index += 1
    data_security_train_pred_Comparison_GPDR.append(data_security_list_Comparison)
    data_security_list_Comparison = []
    line = 0

data_security_train_pred_Comparison_GPDR

    

[[1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1]]

In [54]:
##Privacy Policy Change Matrix:

##Matrix creation:
matrix_data_security_Comparison = pd.DataFrame(columns = ['Phrase',
                                               'Google','Aws','AliExpress','Meta','TikTok','YouTube','Waze','Wix','Bookings','whatsapp'
            ,'apple','wolt','Visa','Mastercard','AirBNB','uber','Spotify','samsung','Wordpress','instagram'
            ,'MacDonalds','FDA','Oracle','Zara','cocacola','Xiaomi','Nasdaq','Walmart'
           ,'AirCanada','Lufthansa','shopify','Netflix','adobe','Starbucks','Shoppers','Decathlon','waltdisney'
            ,'AmericanEagle','lululemon','SAP','JetBrains','MySQLCode','Cadens','EpicGames'
            ,'unitedHealthGroup','Slack','SalesForce','JPMorgan','JohnsonAndJohnson'])
matrix_data_security_Comparison['Phrase'] = [['we secure your data','we are not secure your data']]

##Filling the matrix:
websites_index = 0
line = 0
pred = 0
data_security_train_pred_Comparison_other = []
data_security_list_Comparison = []

for j in sequence_to_classify:
    for i in matrix_data_security_Comparison["Phrase"]:
        a = classifier(str(j), str(i))
        if matrix_data_security_Comparison['Phrase'][line][0] in a["labels"][0]:
            pred = 1
        else:
            pred = 0 
        if pred == 1:
            data_security_list_Comparison.append(line+1)
        line += 1
    websites_index += 1
    data_security_train_pred_Comparison_other.append(data_security_list_Comparison)
    data_security_list_Comparison = []
    line = 0

data_security_train_pred_Comparison_other

    

[[],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [],
 [1],
 [1],
 [1],
 [1],
 [1],
 [],
 [1],
 [1],
 [1],
 [],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [],
 [1],
 [1],
 [1],
 [1],
 [1],
 [],
 [1]]

In [55]:
##Privacy Policy Change Matrix:

##Matrix creation:
matrix_data_security_Comparison = pd.DataFrame(columns = ['Phrase',
                                               'Google','Aws','AliExpress','Meta','TikTok','YouTube','Waze','Wix','Bookings','whatsapp'
            ,'apple','wolt','Visa','Mastercard','AirBNB','uber','Spotify','samsung','Wordpress','instagram'
            ,'MacDonalds','FDA','Oracle','Zara','cocacola','Xiaomi','Nasdaq','Walmart'
           ,'AirCanada','Lufthansa','shopify','Netflix','adobe','Starbucks','Shoppers','Decathlon','waltdisney'
            ,'AmericanEagle','lululemon','SAP','JetBrains','MySQLCode','Cadens','EpicGames'
            ,'unitedHealthGroup','Slack','SalesForce','JPMorgan','JohnsonAndJohnson'])
matrix_data_security_Comparison['Phrase'] = [['we are follow PCI DSS','we are not follow PCI DSS']]

##Filling the matrix:
websites_index = 0
line = 0
pred = 0
data_security_train_pred_Comparison_PCI_DSS = []
data_security_list_Comparison = []

for j in sequence_to_classify:
    for i in matrix_data_security_Comparison["Phrase"]:
        a = classifier(str(j), str(i))
        if matrix_data_security_Comparison['Phrase'][line][0] in a["labels"][0]:
            pred = 1
        else:
            pred = 0 
        if pred == 1:
            data_security_list_Comparison.append(line+1)
        line += 1
    websites_index += 1
    data_security_train_pred_Comparison_PCI_DSS.append(data_security_list_Comparison)
    data_security_list_Comparison = []
    line = 0

data_security_train_pred_Comparison_PCI_DSS

    

[[1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1]]

In [84]:
index = 0
for i in data_security_train_pred_Comparison_2FA:
    if i==[]:
        data_security_train_pred_Comparison_2FA[index]=[0]
    else:
        data_security_train_pred_Comparison_2FA[index]=1
    index +=1

    
index = 0
for i in data_security_train_pred_Comparison_GPDR:
    if i==[]:
        data_security_train_pred_Comparison_GPDR[index]=[0]
    else:
        data_security_train_pred_Comparison_GPDR[index]=1
    index +=1
    
index = 0
for i in data_security_train_pred_Comparison_other:
    if i==[]:
        data_security_train_pred_Comparison_other[index]=[0]
    else:
        data_security_train_pred_Comparison_other[index]=1
    index +=1
    
index = 0
for i in data_security_train_pred_Comparison_PCI_DSS:
    if i==[]:
        data_security_train_pred_Comparison_PCI_DSS[index]=0
    else:
        data_security_train_pred_Comparison_PCI_DSS[index]=1
    index +=1
   

In [85]:
#Data Security Parameter:
index = 0
TP = 0
FP = 0
FN = 0
TN = 0
for i in data_security_train_pred_Comparison_2FA:
    if data_security_train_pred_Comparison_2FA[index]==1 and data_security_train_2FA[index]==1:
        TP +=1
    elif data_security_train_pred_Comparison_2FA[index]==0 and data_security_train_2FA[index]==0:
        TN +=1
    elif data_security_train_pred_Comparison_2FA[index]==0 and data_security_train_2FA[index]==1:
        FN +=1
    else:
        FP +=1
    index +=1

index = 0
for i in data_security_train_pred_Comparison_GPDR:
    if data_security_train_pred_Comparison_GPDR[index]==1 and data_security_train_GDPR[index]==1:
        TP +=1
    elif data_security_train_pred_Comparison_GPDR[index]==0 and data_security_train_GDPR[index]==0:
        TN +=1
    elif data_security_train_pred_Comparison_GPDR[index]==0 and data_security_train_GDPR[index]==1:
        FN +=1
    else:
        FP +=1
    index +=1
    
index = 0
for i in data_security_train_pred_Comparison_other:
    if data_security_train_pred_Comparison_other[index]==1 and data_security_train_other[index]==1:
        TP +=1
    elif data_security_train_pred_Comparison_other[index]==0 and data_security_train_other[index]==0:
        TN +=1
    elif data_security_train_pred_Comparison_other[index]==0 and data_security_train_other[index]==1:
        FN +=1
    else:
        FP +=1
    index +=1
    
index = 0    
for i in data_security_train_pred_Comparison_PCI_DSS:
    if data_security_train_pred_Comparison_PCI_DSS[index]==1 and data_security_train_PCI_DSS[index]==1:
        TP +=1
    elif data_security_train_pred_Comparison_PCI_DSS[index]==0 and data_security_train_PCI_DSS[index]==0:
        TN +=1
    elif data_security_train_pred_Comparison_PCI_DSS[index]==0 and data_security_train_PCI_DSS[index]==1:
        FN +=1
    else:
        FP +=1
    index +=1
    
recall = TP/(TP+FN)
Precision = TP/(TP+FP)
Accuracy = (TP+TN)/(TP+FP+FN+TN) 
F1 = (TP)/(TP+(0.5*(FP+FN)))

print("recall:" , recall)
print("Precision:" , Precision)
print("Accuracy:" , Accuracy)
print("F1 Score:" , F1)

recall: 1.0
Precision: 0.26804123711340205
Accuracy: 0.2755102040816326
F1 Score: 0.42276422764227645
